In [1]:
labels_train = '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/labels_train.csv'
labels_val = '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/labels_val.csv'

image_train_folder = '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_train/images_train'
image_val_folder = '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_val/images_val'

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv(labels_train)
# df['timestamp'] = pd.to_datetime(df['timestamp'], format='%H:%M').dt.time

print(df.head())

       filename timestamp  latitude  longitude  angle  Region_ID
0  img_0000.jpg     15:03    219698     144782    133          2
1  img_0001.jpg     15:05    219844     144621    312          2
2  img_0002.jpg     15:05    219844     144621    359          2
3  img_0003.jpg     17:11    219514     145016    131          2
4  img_0004.jpg     17:00    220182     144211     45          2


In [3]:

import cv2
import os
image = cv2.imread('/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_train/images_train/img_0000.jpg')
print(image.shape)

# in the df change all the file names to full path 
df['filename'] = df['filename'].apply(lambda x: os.path.join('/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_train/images_train/', x))
df.head()

(256, 256, 3)


,filename,timestamp,latitude,longitude,angle,Region_ID
0,/kaggle/input/smai-25-sec-a-project-phase-2-re...,15:03,219698,144782,133,2
1,/kaggle/input/smai-25-sec-a-project-phase-2-re...,15:05,219844,144621,312,2
2,/kaggle/input/smai-25-sec-a-project-phase-2-re...,15:05,219844,144621,359,2
3,/kaggle/input/smai-25-sec-a-project-phase-2-re...,17:11,219514,145016,131,2
4,/kaggle/input/smai-25-sec-a-project-phase-2-re...,17:00,220182,144211,45,2


In [4]:
from torchvision import transforms

train_transforms = transforms.Compose([
    # transforms.RandomResizedCrop(256, scale=(0.8,1.0)),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),

    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
])

train_transforms2 = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8,1.0)),
    transforms.ToTensor(),
    
])

In [5]:
from torch.utils.data import Dataset
from PIL import Image

class MyDataset(Dataset):
    def __init__(self, df, transform=None, validation = False, label_fixing = True):
        self.df = df
        self.transform = transform
        self.double_len = len(df) * 2
        self.validation = validation
        if label_fixing:
            self.df['Region_ID'] = self.df['Region_ID'] - 1

    def __len__(self):
        if self.validation:
            return len(self.df)
        else:
            return self.double_len

    def __getitem__(self, idx):
        row = self.df.iloc[idx % len(self.df)]
        img = Image.open(f"{row.filename}").convert('RGB')
        img = img.resize((224, 224))
        if self.validation:
            transformed_img = transforms.ToTensor()(img)
        elif idx < len(self.df):
            transformed_img = transforms.ToTensor()(img)
        else:
            transformed_img = self.transform(img) if self.transform else img

        label = row.Region_ID
        return transformed_img, label

train_dataset = MyDataset(df, transform=train_transforms) 
print(len(train_dataset))
img_sample, label = train_dataset[len(df) + 4]
print(img_sample.shape)
print(label)

validation_df = pd.read_csv(labels_val)
validation_df['filename'] = validation_df['filename'].apply(lambda x: os.path.join('/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_val/images_val/', x))
validation_dataset = MyDataset(validation_df, transform=train_transforms, validation=True)

print(len(validation_dataset))
img_sample, label = validation_dataset[0]
print(img_sample.shape)
print(label)

13084
torch.Size([3, 224, 224])
1
369
torch.Size([3, 224, 224])
1


In [6]:
import torch

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=16, shuffle=False)
# device = 
print(len(train_loader))
print(len(validation_loader))
# print(device)
print(train_loader.dataset[0][0].shape)
sample_image, sample_label = train_loader.dataset[0]

818
24
torch.Size([3, 224, 224])


In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import os

In [9]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")


import wandb
wandb.login(key = secret_value_0)
# wandb.init(
#     project="smai-project-regionID-classification",
#     name="vit-base-224-finetune",
#     config={
#         "epochs": 40,
#         "batch_size": train_loader.batch_size,
#         "lr": 5e-5,
#         "weight_decay": 0.01,
#         "model": "ViT-base-patch16-224",
#         "num_labels": 15
#     }
# )




wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: v4ishnavi (v4ishnavi-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# # fully finetuning a vit v
# import torch
# from transformers import ViTForImageClassification

# num_labels = 15  
# model = ViTForImageClassification.from_pretrained(
#     'google/vit-base-patch16-224-in21k',
#     num_labels=num_labels
# )
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# # Watch model gradients and parameter updates
# wandb.watch(model, log="all", log_freq=100)
# from torch import nn, optim
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

In [ ]:
# import torch
# import copy

# class EarlyStopping:
#     def __init__(self, patience=5, min_delta=0.0, verbose=False):
#         self.patience = patience
#         self.min_delta = min_delta
#         self.verbose = verbose
#         self.counter = 0
#         self.best_loss = float('inf')
#         self.best_model_state = None
#         self.early_stop = False

#     def __call__(self, val_loss, model):
#         if val_loss < self.best_loss - self.min_delta:
#             self.best_loss = val_loss
#             self.best_model_state = copy.deepcopy(model.state_dict())
#             self.counter = 0
#             if self.verbose:
#                 print(f"Validation loss improved to {val_loss:.4f}.")
#         else:
#             self.counter += 1
#             if self.verbose:
#                 print(f"No improvement in validation loss for {self.counter} epoch(s).")
#             if self.counter >= self.patience:
#                 self.early_stop = True
#                 if self.verbose:
#                     print("Early stopping triggered.")

# num_epochs = 40

In [ ]:
# # Initialize early stopping
# early_stopping = EarlyStopping(patience=3, min_delta=0.001, verbose=True)
# from tqdm import tqdm 
# for epoch in range(num_epochs):
#     # Training phase
#     model.train()
#     train_loss = 0.0
#     correct_train = 0
#     total_train = 0

#     for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
#         inputs = images.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(pixel_values=inputs)
#         loss = criterion(outputs.logits, labels)
#         loss.backward()
#         optimizer.step()

#         train_loss += loss.item() * inputs.size(0)
#         _, predicted = outputs.logits.max(1)
#         correct_train += predicted.eq(labels).sum().item()
#         total_train += labels.size(0)

#     avg_train_loss = train_loss / total_train
#     train_accuracy = correct_train / total_train

#     # Evaluation phase
#     model.eval()
#     val_loss = 0.0
#     correct_val = 0
#     total_val = 0

#     with torch.no_grad():
#         for images, labels in tqdm(validation_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
#             inputs = images.to(device)
#             labels = labels.to(device)

#             outputs = model(pixel_values=inputs)
#             loss = criterion(outputs.logits, labels)

#             val_loss += loss.item() * inputs.size(0)
#             _, predicted = outputs.logits.max(1)
#             correct_val += predicted.eq(labels).sum().item()
#             total_val += labels.size(0)

#     avg_val_loss = val_loss / total_val
#     val_accuracy = correct_val / total_val

#     print(f"Epoch {epoch+1}/{num_epochs}")
#     print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
#     print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
#     wandb.log({
#         "epoch": epoch + 1,
#         "train_loss": avg_train_loss,
#         "train_accuracy": train_accuracy,
#         "val_loss": avg_val_loss,
#         "val_accuracy": val_accuracy
#     })

#     # Check early stopping condition
#     early_stopping(avg_val_loss, model)

#     if early_stopping.early_stop:
#         print("Early stopping triggered. Restoring best model weights.")
#         model.load_state_dict(early_stopping.best_model_state)
#         break
# torch.save(model.state_dict(), "vit_finetuned.pth")
# wandb.save("vit_finetuned.pth")


trying yolo ...

In [10]:
# for yolo we will have to restructure the files 
import os
import shutil
import pandas as pd
wandb.init(
    project="smai-project-regionID-classification",
    name="yolov11-v2-x",
    config={
        "epochs": 50,
        "batch_size": 32,
        "model": "yolov11x",
        "num_labels": 15
    }
)
df['Region_ID'] = df['Region_ID'].astype(str)
validation_df['Region_ID'] = validation_df['Region_ID'].astype(str)

base_dir = "/kaggle/working/yolo_dataset"
all_labels = sorted(set(df['Region_ID'].unique()) | set(validation_df['Region_ID'].unique()))
all_labels_str = [str(l) for l in all_labels]

for split_df, split in [(df, 'train'), (validation_df, 'val')]:
    for label in all_labels_str:
        os.makedirs(os.path.join(base_dir, split, label), exist_ok=True)

def copy_images(split_df, split_name):
    for _, row in split_df.iterrows():
        src = row['filename']
        label = row['Region_ID']
        dst = os.path.join(base_dir, split_name, label, os.path.basename(src))
        shutil.copy(src, dst)

copy_images(df, 'train')
copy_images(validation_df, 'val')


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [11]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [12]:
import yaml
data_yaml = {
    'path': base_dir,
    'train': 'train',
    'val': 'val',
    'nc': len(all_labels_str),
    'names': all_labels_str
}

with open(os.path.join(base_dir, 'data.yaml'), 'w') as f:
    yaml.dump(data_yaml, f)

In [13]:
!cat /kaggle/working/yolo_dataset/data.yaml

names:
- '0'
- '1'
- '10'
- '11'
- '12'
- '13'
- '14'
- '2'
- '3'
- '4'
- '5'
- '6'
- '7'
- '8'
- '9'
nc: 15
path: /kaggle/working/yolo_dataset
train: train
val: val


In [ ]:
from ultralytics import YOLO, settings
settings.update({'wandb': True}) 

os.environ['WANDB_PROJECT'] = 'smai-project-regionID-classification'
os.environ['WANDB_NAME'] = 'yolo11m-bigger-classification-run'
model = YOLO('yolo11x-cls.pt')  
dataset_path = '/kaggle/working/yolo_dataset'
model.train(
    data=dataset_path,
    epochs=50,
    imgsz=256,
    batch=32,
    patience=10
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 56.9M/56.9M [00:02<00:00, 27.8MB/s]


Ultralytics 8.3.127 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_dataset, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=

train: Scanning /kaggle/working/yolo_dataset/train... 6542 images, 0 corrupt: 100%|██████████| 6542/6542 [00:08<00:00, 730.86it/s] 

train: /kaggle/working/yolo_dataset/train/13/img_5741.jpg: corrupt JPEG restored and saved
train: /kaggle/working/yolo_dataset/train/13/img_5742.jpg: corrupt JPEG restored and saved
train: /kaggle/working/yolo_dataset/train/13/img_5743.jpg: corrupt JPEG restored and saved
train: /kaggle/working/yolo_dataset/train/13/img_5744.jpg: corrupt JPEG restored and saved
train: /kaggle/working/yolo_dataset/train/13/img_5745.jpg: corrupt JPEG restored and saved
train: /kaggle/working/yolo_dataset/train/13/img_5746.jpg: corrupt JPEG restored and saved
train: /kaggle/working/yolo_dataset/train/13/img_5747.jpg: corrupt JPEG restored and saved
train: /kaggle/working/yolo_dataset/train/13/img_5748.jpg: corrupt JPEG restored and saved
train: /kaggle/working/yolo_dataset/train/13/img_5749.jpg: corrupt JPEG restored and saved
train: /kaggle/working/yolo_dataset/train/13/img_5750.jpg: corrupt JPEG restored and saved
train: /kaggle/working/yolo_dataset/train/13/img_5751.jpg: corrupt JPEG restored and saved


val: Scanning /kaggle/working/yolo_dataset/val... 369 images, 0 corrupt: 100%|██████████| 369/369 [00:00<00:00, 731.52it/s]


val: /kaggle/working/yolo_dataset/val/13/img_0324.jpg: corrupt JPEG restored and saved
val: /kaggle/working/yolo_dataset/val/13/img_0325.jpg: corrupt JPEG restored and saved
val: /kaggle/working/yolo_dataset/val/13/img_0326.jpg: corrupt JPEG restored and saved
val: New cache created: /kaggle/working/yolo_dataset/val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000526, momentum=0.9) with parameter groups 82 weight(decay=0.0), 83 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 0 dataloader workers
Logging results to runs/classify/train
Starting training for 50 epochs...

      Epoch    GPU_mem       loss  Instances       Size


       1/50         0G      2.846         32        256:   0%|          | 1/205 [00:26<1:28:48, 26.12s/it]


  0%|          | 0.00/755k [00:00<?, ?B/s]
100%|██████████| 755k/755k [00:00<00:00, 4.13MB/s]
       1/50         0G      2.803         32        256:   1%|          | 2/205 [00:50<1:24:54, 25.10s/it]

In [ ]:
import pandas as pd
import os
import numpy as np
from ultralytics import YOLO

# Load model
model = YOLO('/kaggle/working/runs/classify/train4/weights/best.pt')

# Paths
val_image_dir = '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_val/images_val'
labels_val_path = '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/labels_val.csv'

# Load label file
labels_df = pd.read_csv(labels_val_path)
labels_df['filename'] = labels_df['filename'].apply(lambda x: os.path.join(val_image_dir, x))
print(labels_df.head())

# Create a mapping from YOLO class indices to Region_ID values
# First, get the unique region IDs from your validation data
unique_region_ids = sorted(labels_df['Region_ID'].unique())
print(f"Unique Region IDs: {unique_region_ids}")

# Get the class names from your YAML file to understand YOLO's ordering
# Or use this logic which shows the order YOLO uses
class_folders = ['0', '1', '10', '11', '12', '13', '14', '2', '3', '4', '5', '6', '7', '8', '9']
# The actual indices YOLO assigns (alphabetical order)
yolo_indices = list(range(len(class_folders)))
# Map from YOLO index to class folder name
idx_to_folder = {idx: folder for idx, folder in zip(yolo_indices, class_folders)}
print(f"YOLO index to folder mapping: {idx_to_folder}")

# Create a mapping from YOLO prediction to actual Region_ID
# If your folder names match the Region_IDs directly
idx_to_region_id = {idx: int(folder) for idx, folder in idx_to_folder.items()}
print(f"YOLO index to Region_ID mapping: {idx_to_region_id}")

# Predict on val images
region_ids = []
correct_count = 0
total_count = 0

for index, row in labels_df.iterrows():
    img_path = row['filename']
    true_region_id = row['Region_ID']
    
    result = model.predict(img_path, verbose=False)
    pred_class_idx = int(result[0].probs.top1)  # Get top prediction index
    
    # Map the predicted class index to the correct Region_ID
    pred_region_id = idx_to_region_id[pred_class_idx] + 1
    region_ids.append(pred_region_id)
    
    # Track accuracy
    if pred_region_id == true_region_id:
        correct_count += 1
    total_count += 1

print(f"Validation accuracy: {correct_count/total_count:.4f}")

# Prepare validation part of submission
val_df = pd.DataFrame({
    'id': labels_df.index,
    'Region_ID': region_ids,
    # 'Real_RID': labels_df['Region_ID']
})

# Prepare test part (ID 369 to 737)
# You may want to predict on test images properly instead of using dummy values
# test_image_dir = '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_test/images_test'
# test_filenames = sorted(os.listdir(test_image_dir))

# test_region_ids = []
# for filename in test_filenames:
#     img_path = os.path.join(test_image_dir, filename)
#     result = model.predict(img_path, verbose=False)
#     pred_class_idx = int(result[0].probs.top1)
#     pred_region_id = idx_to_region_id[pred_class_idx]
#     test_region_ids.append(pred_region_id)
lenh = 369
test_df = pd.DataFrame({
    'id': list(range(369, 369 + lenh)),
    'Region_ID': [1] * lenh,
    # 'Real_RID': [1] * lenh  # Placeholder
})

# Combine
submission_df = pd.concat([val_df, test_df], ignore_index=True)
print(submission_df.head())

# Save submission
submission_filename = '2022102070.csv'
submission_df.to_csv(submission_filename, index=False)
print(f"Submission file saved as: {submission_filename}")

In [ ]:
wandb.init(
    project="smai-project-regionID-classification",  # Change as needed
    name="upload-best-model",
    job_type="model-upload"
)
artifact = wandb.Artifact(
    name="yolo-best-classifier",  # A name you'll use to refer to this model later
    type="model"
)
artifact = wandb.Artifact(
    name = "yolo-results-csv",
    type = "results"
)
artifact.add_file("/kaggle/working/runs/classify/train4/weights/best.pt")  # or your renamed version

wandb.log_artifact(artifact)

wandb.finish()

In [ ]:
wandb.finish()

In [ ]:
# wandb.login()
# wandb.init(
#     project="smai-project-regionID-classification",
#     name="offline-efficientnet-partial-finetune-07-save-model",
#     settings=wandb.Settings(init_timeout=180)
# )
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# # Load pretrained model
# model_eb2 = models.efficientnet_b2(pretrained=True)
# num_classes = 15
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Replace final layer in classifier
# in_features = model_eb2.classifier[1].in_features
# model_eb2.classifier[1] = nn.Linear(in_features, num_classes)
# model_eb2 = model_eb2.to(device)

# # Freeze early layers (first 70% of the model)
# total_blocks = len(model_eb2.features)
# freeze_blocks = int(total_blocks * 0.7)  # Freeze first 70% of feature blocks

# # Freeze early layers
# for i, block in enumerate(model_eb2.features):
#     if i < freeze_blocks:
#         for param in block.parameters():
#             param.requires_grad = False
#     else:
#         for param in block.parameters():
#             param.requires_grad = True

# # Always train the classifier layers
# for param in model_eb2.classifier.parameters():
#     param.requires_grad = True

# # Use a lower learning rate for the non-frozen layers compared to the classifier
# optimizer = optim.Adam([
#     {'params': model_eb2.features[freeze_blocks:].parameters(), 'lr': 0.0005},
#     {'params': model_eb2.classifier.parameters(), 'lr': 0.001}
# ])

# # Define the learning rate scheduler
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# criterion = nn.CrossEntropyLoss()
# num_epochs = 40
# best_val_loss = float('inf')
# patience = 5  # Increased patience for better convergence
# patience_counter = 0
# best_model_state = None

# from tqdm import tqdm

# # Print trainable parameters to verify setup
# trainable_params = sum(p.numel() for p in model_eb2.parameters() if p.requires_grad)
# total_params = sum(p.numel() for p in model_eb2.parameters())
# print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%} of total)")

# for epoch in range(num_epochs):
#     print(f"\nEpoch {epoch+1}/{num_epochs}")
#     print("-" * 30)
    
#     # ---------- Training Phase ----------
#     model_eb2.train()
#     train_loss = 0.0
#     train_correct = 0
#     train_total = 0
    
#     for images, labels in tqdm(train_loader, desc="Training", leave=False):
#         images = images.to(device)
#         labels = labels.to(device)
        
#         optimizer.zero_grad()
#         outputs = model_eb2(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         train_loss += loss.item() * images.size(0)
#         _, predicted = torch.max(outputs, 1)
#         train_total += labels.size(0)
#         train_correct += (predicted == labels).sum().item()
    
#     avg_train_loss = train_loss / train_total
#     train_accuracy = 100 * train_correct / train_total
    
#     # ---------- Validation Phase ----------
#     model_eb2.eval()
#     val_loss = 0.0
#     val_correct = 0
#     val_total = 0
    
#     with torch.no_grad():
#         for images, labels in tqdm(validation_loader, desc="Validation", leave=False):
#             images = images.to(device)
#             labels = labels.to(device)
            
#             outputs = model_eb2(images)
#             loss = criterion(outputs, labels)
            
#             val_loss += loss.item() * images.size(0)
#             _, predicted = torch.max(outputs, 1)
#             val_total += labels.size(0)
#             val_correct += (predicted == labels).sum().item()
    
#     avg_val_loss = val_loss / val_total
#     val_accuracy = 100 * val_correct / val_total
    
#     # Update learning rate scheduler
#     scheduler.step(avg_val_loss)
    
#     # ---------- Epoch Summary ----------
#     print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.2f}%")
#     print(f"Val   Loss: {avg_val_loss:.4f} | Val   Acc: {val_accuracy:.2f}%")
    
#     wandb.log({
#         'avg_val_loss': avg_val_loss,
#         'val_accuracy': val_accuracy,
#         'avg_train_loss': avg_train_loss,
#         'train_accuracy': train_accuracy,
#         'epoch': epoch,
#         'learning_rate': optimizer.param_groups[0]['lr']
#     })
    
#     if avg_val_loss < best_val_loss:
#         best_val_loss = avg_val_loss
#         patience_counter = 0
#         best_model_state = model_eb2.state_dict()
#         print("✅ Validation loss improved. Saving model.")
#     else:
#         patience_counter += 1
#         print(f"⚠️ No improvement in val loss. Patience {patience_counter}/{patience}")
#         if patience_counter >= patience:
#             print("⛔ Early stopping triggered.")
#             break

# torch.save(best_model_state, 'best_model_partial_finetune_eb2.pth')


# artifact = wandb.Artifact('efficientnet-best-model', type='model')
# artifact.add_file('best_model_partial_finetune_eb2.pth')
# wandb.log_artifact(artifact)

# import pandas as pd
# import os
# import numpy as np
# import torch
# import torch.nn as nn
# from torchvision import models, transforms
# from PIL import Image
# import wandb

# # Initialize wandb
# wandb.init(
#     project="smai-project-regionID-classification",
#     name="efficientnet-inference",
# )

# # Define image transformations (must match what was used during training)
# transform = transforms.Compose([
#     transforms.Resize(224),
#     transforms.ToTensor(),
# ])

# # Paths
# val_image_dir = '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_val/images_val'
# labels_val_path = '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/labels_val.csv'
# test_image_dir = '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_test/images_test'

# # Load label file
# labels_df = pd.read_csv(labels_val_path)
# labels_df['filename'] = labels_df['filename'].apply(lambda x: os.path.join(val_image_dir, x))
# print(labels_df.head())

# # Load the EfficientNet model
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_eb2 = models.efficientnet_b2(pretrained=False)
# num_classes = 15
# in_features = model_eb2.classifier[1].in_features
# model_eb2.classifier[1] = nn.Linear(in_features, num_classes)

# # Load the saved model weights
# model_eb2.load_state_dict(torch.load('best_model_partial_finetune_eb2.pth'))
# model_eb2 = model_eb2.to(device)
# model_eb2.eval()

# print("Model loaded successfully")

# def predict_region_id(img_path, model):
#     try:
#         image = Image.open(img_path).convert('RGB')
#         image_tensor = transform(image).unsqueeze(0).to(device)
        
#         with torch.no_grad():
#             outputs = model(image_tensor)
#             _, predicted = torch.max(outputs, 1)
            
#         pred_region_id = predicted.item() + 1
#         return pred_region_id
#     except Exception as e:
#         print(f"Error processing {img_path}: {e}")
#         return 1  # Default value in case of error

# # Predict on validation images
# region_ids = []
# correct_count = 0
# total_count = 0

# print("Processing validation images...")
# for index, row in labels_df.iterrows():
#     img_path = row['filename']
#     true_region_id = row['Region_ID']
    
#     pred_region_id = predict_region_id(img_path, model_eb2)
#     region_ids.append(pred_region_id)
    
#     # Track accuracy
#     if pred_region_id == true_region_id:
#         correct_count += 1
#     total_count += 1
    
#     if index % 50 == 0:
#         print(f"Processed {index} validation images")

# validation_accuracy = correct_count/total_count
# print(f"Validation accuracy: {validation_accuracy:.4f}")

# # Prepare validation part of submission
# val_df = pd.DataFrame({
#     'id': labels_df.index,
#     'Region_ID': region_ids
# })
# val_df2 = pd.DataFrame({
#     'id': labels_df.index,
#     'Region_ID': region_ids,
#     'Real_RID': labels_df['Region_ID']
# })

# # Process test images
# # print("Processing test images...")
# # test_filenames = sorted(os.listdir(test_image_dir))
# # test_region_ids = []

# # for filename in test_filenames:
# #     img_path = os.path.join(test_image_dir, filename)
# #     pred_region_id = predict_region_id(img_path, model_eb2)
# #     test_region_ids.append(pred_region_id)
    
# #     if len(test_region_ids) % 50 == 0:
# #         print(f"Processed {len(test_region_ids)} test images")
# lenh = 369
# # Prepare test part of submission
# test_df = pd.DataFrame({
#     'id': list(range(369, 369 + lenh)),
#     'Region_ID': [1]*lenh
# })
# test_df2 = pd.DataFrame({
#     'id': list(range(369, 369 + lenh)),
#     'Region_ID': [1]*lenh,
#     'Real_RID':[1]*lenh
# })
# # Combine validation and test predictions
# submission_df = pd.concat([val_df, test_df], ignore_index=True)
# submission_df2 = pd.concat([val_df2, test_df2], ignore_index=True)
# print(submission_df.head())

# # Save submission file
# submission_filename = '2022102070.csv'
# submission_df.to_csv(submission_filename, index=False)
# print(f"Submission file saved as: {submission_filename}")

# submission_filename2 = '2022102070_withcorrect.csv'
# submission_df2.to_csv(submission_filename2, index=False)
# print(f"Submission file saved as: {submission_filename2}")

# # Log the CSV file to wandb
# artifact = wandb.Artifact('prediction_results', type='dataset')
# artifact.add_file(submission_filename)
# wandb.log_artifact(artifact)
# artifact = wandb.Artifact('prediction_results_with_real', type='dataset')
# artifact.add_file(submission_filename2)
# wandb.log_artifact(artifact)

# # Log the validation accuracy to wandb
# wandb.log({"validation_accuracy": validation_accuracy})